In [2]:
import numpy as np

with open('../query_gender.train') as fp:    
    i = 1
    train = []
    while i >= 0:
        line = fp.readline()
        train.append(line)
        i -= 1

del train[0]

In [79]:
len(train)

1

In [163]:
def create_ngrams(n, instances):
    i = 0
    for inst in instances:
        for sentence in inst:
            newsent = sentence[:]
            
            while n > 1:
                ngrams = zip(*[sentence[i:] for i in range(n)])
                for gram in ngrams:
                    newsent.append(gram)
                n -= 1
            
            print(newsent) 
            print
            
            i += 1
    
            if i >= 1:
                break

In [28]:
def create_char_ngrams(n, instances):
    for inst in instances:
        for sentence in inst:
            for word in sentence:
                char_ngrams = [word[i:i+n] for i in range(len(word)-n+1)]
                    
                #for gram in char_ngrams:
                    #sentence.append(gram)
                        
                print(word)
    print()

In [217]:
words =  []
labels = []
documents = []
# whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ 0123456789 \t \n')
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ \t \n')

# loop through each instance in training data, gets labels
for x in train:
    i = 0
    inst = []
    label = x[0:10]
    if label[0:9] != '__label__':
        print("ERROR in label creation")
        break
    else:
        labels.append(float(label[-1]))

    sent = []
    word = ''
    for w in x[10:]:
        if w in whitelist:
            if w == '\t':
                inst.append(sent)
                sent = []
                word = ''
                i += 1
            elif w != ' ':
                word = word + w
            else:
                if "http" not in word and word != "RT" and word != "rt":
                    if word != '':
                        sent.append(word.lower())
                    word = ''
                else:
                    word = ''
    documents.append(inst)
    
print len(documents)

1


In [110]:
## %pprint
#documents

In [178]:
create_ngrams(3, documents)

In [3]:
def create_instances_and_labels(dataset):
        words =  []
        labels = []
        documents = []
        #whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ 0123456789 \t \n')
        whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ \t \n')

        # loop through each instance in training data, gets labels
        for x in dataset:
            i = 0
            inst = ''
            label = x[0:10]
            if label[0:9] != '__label__':
                print("ERROR in label creation")
                break
            else:
                labels.append(float(label[-1]))
                
            sent = ''
            word = ''
            for w in x[10:]:
                if w in whitelist:
                    if w == '\t':
                        #inst.append(sent)
                        inst = inst + '\t' + sent
                        sent = ''
                        word = ''
                        i += 1
                    elif w != ' ':
                        word = word + w
                    else:
                        if "http" not in word and word != "RT" and word != "rt":
                            sent = sent + ' ' + word.lower()
                            word = ''
                        else:
                            word = ''
            
            documents.append(inst)
            
        return documents

In [219]:
documents

[[['cesecimi', 'im', 'good', 'dear', 'and'], ['i', 'have', 'the', 'best', 'name', 'for', 'kiss', 'daniel'], ['i', 'have', 'the', 'best', 'name', 'for', 'kiss'], ['cesecimi', 'being', 'a', 'while', 'my'], ['get', 'ready', 'spend', 'on', 'datamake', 'telecom', 'networks', 'amp', 'organizers', 'richer', 'amp', 'lucky', 'dude', 'or', 'babe', 'famous', 'amp', 'richer', 'yl', 'ure', 'still', 'hustling'], ['armed', 'robbers', 'were', 'trying', 'to', 'rob', 'me', 'so', 'i', 'started', 'running', 'while', 'running', 'i', 'sighted', 'sars', 'i', 'turned', 'back', 'amp', 'ran', 'to', 'd', 'thieves'], ['so', 'you', 'treat', 'me', 'like', 'a', 'modern', 'slave', 'mr', 'jailer'], ['ufefeman', 'hello'], ['please', 'i', 'need', 'strong', 'men', 'to', 'define', 'magun', 'feels', 'good', 'to', 'be', 'back', 'again', 'ive', 'missed'], ['newprofilepic'], ['if', 'you', 'like', 'load', 'it', 'thinking', 'its', 'recharge', 'card', 'its', 'the', 'two', 'days', 'of', 'public', 'holidays', 'naija', 'and'], ['re

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import FeatureUnion

documents = create_instances_and_labels(train)
    
word_ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))
char_ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))

X_word_ngrams = word_ngram_vectorizer.fit_transform(documents)
X_char_ngrams = char_ngram_vectorizer.fit_transform(documents)

#combined_features = FeatureUnion([('CountVectorizer1', X_word_ngrams),('CountVectorizer2', X_char_ngrams)])


In [199]:
X_char_ngrams

<1x565 sparse matrix of type '<type 'numpy.int64'>'
	with 565 stored elements in Compressed Sparse Row format>

In [203]:
word_ngram_vectorizer = CountVectorizer(ngram_range=(1, 2), analyzer=words_and_char_bigrams)


NameError: name 'words_and_char_bigrams' is not defined

In [6]:
# https://stackoverflow.com/questions/21600196/how-do-i-use-sklearn-countvectorizer-with-both-word-and-char-analyzer-pyt

from pprint import pprint
import re

x = ['this is a foo bar', 'you are a foo bar black sheep']

documents = create_instances_and_labels(train)

def words_and_char_bigrams(text):
    words = re.findall(r'\w{3,}', text)
    for w in words:
        yield w
        for i in range(len(w) - 2):
            yield w[i:i+2]
            
def words_and_char_ngrams(text):
    words = re.findall(r'\w{3,}', text)  # {3,} "3 OR MORE"
    for w in words:
        numgrams = 3
        yield w
        while numgrams > 1:
            for i in range(len(w) - numgrams):
                yield w[i:i+numgrams]
            numgrams -= 1
            
           
v = CountVectorizer(analyzer=words_and_char_ngrams, ngram_range=(2,1))
features = v.fit(x)
bag = features.transform(x)

print bag.shape
pprint(features.vocabulary_)

(2, 24)
{'ac': 0,
 'ar': 1,
 'are': 2,
 'ba': 3,
 'bar': 4,
 'bl': 5,
 'bla': 6,
 'black': 7,
 'ee': 8,
 'fo': 9,
 'foo': 10,
 'he': 11,
 'hee': 12,
 'hi': 13,
 'la': 14,
 'lac': 15,
 'sh': 16,
 'she': 17,
 'sheep': 18,
 'th': 19,
 'thi': 20,
 'this': 21,
 'yo': 22,
 'you': 23}


In [280]:
# access users' descriptions

import tweepy
import json
from tweepy import OAuthHandler
from ACCESS_TOKENS_DONTSHARE import *   # To get twitter api authentication creds

        
CONSUMER_KEY = get_consumer_key()
CONSUMER_SECRET = get_consumer_secret()
ACCESS_KEY = get_access_token()
ACCESS_SECRET = get_access_secret()

auth = OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
api = tweepy.API(auth)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

#search
api = tweepy.API(auth)